### Commands

In [1]:
from firecloud import api as firecloud_api
import fc_interface

In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
### Parameters
namespace = "nci-mimoun-bi-org"
workspace = "CCLF_TSCA"
paths_to_samples_info = "paths_to_batches_info.xlsx"
google_bucket_id = "fc-35446f22-ea37-483a-bd6c-5e9fc56851ff"
batches_info = pd.read_excel(paths_to_samples_info)

#### Deletions

In [ ]:
### Delete unnecessary attributes in samples
attrs = ["annotate_variants_vcf", "annotate_variants_vcf_index", "oncotated_maf", "mutect2_vcf_index", "match_normal_oncotated_maf", "gnomad_vcf", "merged_vcfs", "match_primary_tumor_bam_file"]
for idx, row in all_samples.iterrows():
    fc_interface.delete_entity_attributes(namespace, workspace, "sample", row['entity:sample_id'], attrs)

In [ ]:
### Delete method repostory
firecloud_api.delete_repository_method('tsca', 'CreatePoN', 15)

In [ ]:
### Delete method repository
firecloud_api.delete_repository_method('tsca', 'Mutect2_for_Normal', 1)

In [ ]:
### Delete multiple method repositories
for i in range(10, 23):
    firecloud_api.delete_repository_method('tsca', 'CreatePoN', i)

In [ ]:
### Delete method config
firecloud_api.delete_workspace_config(namespace, workspace, "tsca", "Mutect2_for_Normal")

#### Pair creation

In [ ]:
### Create pairs
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

In [ ]:
### Upload pairs
res = fc_interface.upload_pairs(namespace, workspace, clean_pairs_list)

In [ ]:
# ### Delete unnecessary attributes in pairs
# ### NOT WOKING
# all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
# pairs_list = fc_interface.create_pairs_list(all_samples)
# attrs = ["__annotate_variants_vcf", "__annotate_variants_vcf_index", \
#          "__scattered_intervals", "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2", \
#          "__tmp_mutect_vcf2", "_tmp_mutect_vcf_index2"]
# for idx, row in pairs_list.iterrows():
#     fc_interface.delete_entity_attributes(namespace, workspace, "pair", row['entity:pair_id'], attrs)

In [ ]:
### Upload metadata for given batch
fc_interface.update_batch_metadata('TSCA21', \
                                  '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                                      paths_to_samples_info, \
                                        namespace, workspace, google_bucket_id)

In [ ]:
### Delete sample_set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### All paths to batch info
# pd.read_excel('paths_to_batches_info.xlsx').path_to_samples_info.tolist()

In [ ]:
### Create cumulative PoN (all batches)
batches_info = pd.read_excel(paths_to_samples_info)
PoN = fc_interface.create_panel_of_normals(batches_info.path_to_samples_info.tolist(), -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'CumPoN21', namespace, workspace)

In [ ]:
### Create PoN from normals of only one batch (no samples from other batches)
batches_info = pd.read_excel(paths_to_samples_info)
tsca21_path = batches_info.path_to_samples_info.tolist()[-1]
PoN = fc_interface.create_panel_of_normals([tsca21_path], -1, 'PoN_TSCA21_Normals')
fc_interface.upload_pon(PoN, 'PoN_TSCA21_Normals', namespace, workspace)

In [ ]:
### Delete pair set
fc_interface.delete_pair_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Delete sample set
fc_interface.delete_sample_set(namespace, workspace, 'TSCA1213')

In [ ]:
### Download method configs
fc_interface.download_method_configs(namespace, workspace)

In [ ]:
### Upload pairsets
tn_pairsets, tp_pairsets = fc_interface.create_pairsets(all_samples, clean_pairs_list)
fc_interface.upload_pairsets(namespace, workspace, tn_pairsets, "TN")
fc_interface.upload_pairsets(namespace, workspace, tp_pairsets, "TP")

In [ ]:
### Delete all pairs
for idx, row in clean_pairs_list.iterrows():
    res = fc_interface.delete_pair(namespace, workspace, row['entity:pair_id'])
    print(res)

In [ ]:
all_samples = fc_interface.get_samples_multiple_batches(paths_to_samples_info, google_bucket_id)
pairs_list = fc_interface.create_pairs_list(all_samples)
blacklist = ["DW039-Tumor-SM-DB2IF"]
clean_pairs_list = pairs_list[ ~pairs_list['case_sample_id'].isin(blacklist)]

### Update pair attributes
update_pair_attrs(namespace, workspace, clean_pairs_list, attrs=['control_sample_tsca_id'])

In [ ]:
### Create PoN from all normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [6]:
### Create PoN with normals from batch + other normals
batches_info = pd.read_excel(paths_to_samples_info)
all_paths = batches_info.path_to_samples_info.tolist()
# PoN = fc_interface.create_panel_of_normals(all_paths, -1, 'Cum_PoN_21_all')
# fc_interface.upload_pon(PoN, 'Cum_PoN_21_all', namespace, workspace)

In [4]:
# batch_id = 'TSCA21'
# batch_path = batches_info.loc[batches_info.tsca_id==batch_id, 'path_to_samples_info'].item()
# pd.read_table(batch_path)
# batch_id, paths, N, name

In [21]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [ ]:
### Update WDLs
fc_interface.download_remote_wdls(namespace, workspace)

In [336]:
### Prepare cohorts for export to FC
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [358]:
# # Delete cohorts
# cohort_codes = pd.read_table('cohort_files/cohort_names_dictionary.txt', header=None)
# for coh_code in cohort_codes.loc[:, 1].tolist():
#     res = fc_interface.delete_sample_set(namespace, workspace, coh_code)

In [338]:
# ### Upload metadata for given batch
# fc_interface.update_batch_metadata('TSCA21', \
#                                   '/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
#                                       paths_to_samples_info, \
#                                         namespace, workspace, google_bucket_id)

65 Participants in this batch
95 Samples in this batch


In [361]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [359]:
# Cohorts
cohorts = fc_interface.prepare_cohorts_for_metadata_export(paths_to_batches_info, google_bucket_id)

In [66]:
### Create panel of normals from batch
pon = fc_interface.create_panel_of_normals_from_batch(batch_id, paths_to_samples_info, N=20).shape

In [78]:
### Create panel of normals from batch
for i, row in batches_info.iterrows():
    pon, name = fc_interface.create_panel_of_normals_from_batch(row['tsca_id'], paths_to_samples_info, N=20)
    fc_interface.upload_pon(pon, name, namespace, workspace)

In [ ]:
# Retrieve samples before export to FC
tsca21 = \
    fc_interface.\
        prepare_batch_samples_for_metadata_export('/xchip/clf/seq_data/processed_for_fh/tsca21_201708_SN0128552/tsca21_201708_SN0128552.import_samples.txt', \
                            'TSCA21', google_bucket_id)

In [33]:
### Prepare cohorts for export to FC
cohorts = fc_interface.\
            prepare_cohorts_for_metadata_export(paths_to_samples_info, google_bucket_id, \
                                               blacklist=["CCLF_AA1012-Tumor-SM-F67DF"])

cohorts_sample_set_metadata = "cohort_files/fc_upload_sample_set_cohorts.txt"
res = fc_interface.upload_entities_from_tsv(namespace, workspace, cohorts_sample_set_metadata)

In [8]:
# Export metadata
(r1, r2, r3, r4, r5, r6) = fc_interface.export_batch_metadata_to_fc('TSCA21', namespace, workspace)

In [34]:
### Cohort of all tumors
res = fc_interface.upload_cohort_all_tumors(paths_to_samples_info, google_bucket_id, \
                                      'Cum_Tumors_22_all', namespace, workspace, ['DW039-Tumor-SM-DB2IF'])

In [12]:
# fc_interface.merge_walkupseq_files(paths_to_samples_info)

(1045, 14)

---

---

In [ ]:
tsca_id = "TSCA22"
path_to_batch_samples_info = batches_info.loc[batches_info.tsca_id==tsca_id, 'path_to_samples_info'].item()

In [28]:
# fc_interface.prepare_patients_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_sample_set_for_metadata_export(path_to_batch_samples_info, tsca_id)
# fc_interface.prepare_batch_samples_for_metadata_export(path_to_batch_samples_info, tsca_id, google_bucket_id)
fc_interface.prepare_cohorts_for_metadata_export()

,entity:sample_id,participant_id,clean_bam_file_capture,external_id_validation,aggregation_product_name_validation,bsp_sample_id_validation,stock_sample_id_validation,sample_type,picard_aggregation_type_validation,processed_subtype_validation,source_subtype_validation,squid_sample_id_validation,tumor_subtype,short_letter_code,bam_filename,bai_filename,clean_bai_file_capture,tsca_id
0,AA53-Tumor-SM-GL7LQ,AA53,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA53T_3_OPAC_p5,TSCA Rapid Cancer Detection Panel v2,SM-GL7LQ,SM-GIR74,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,AA53T_3_OPAC_p5,Metastatic,TM,2_AA53T_3_OPAC_p5_HGM3CADXY.2.aligned.duplicat...,2_AA53T_3_OPAC_p5_HGM3CADXY.2.aligned.duplicat...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
1,AA61-Tumor-SM-GL7LF,AA61,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,AA61T_OPAC_p8,TSCA Rapid Cancer Detection Panel v2,SM-GL7LF,SM-GIR6S,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,AA61T_OPAC_p8,Metastatic,TM,2_AA61T_OPAC_p8_HGM3CADXY.2.aligned.duplicates...,2_AA61T_OPAC_p8_HGM3CADXY.2.aligned.duplicates...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
2,CCLF_AA1005-Tumor-SM-GL7NT,CCLF_AA1005,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,CCLF_AA1005T_CM_p11,TSCA Rapid Cancer Detection Panel v2,SM-GL7NT,SM-GIVWN,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",CCLF_AA1005T_CM_p11,Metastatic,TM,2_CCLF_AA1005T_CM_p11_HGM3CADXY.2.aligned.dupl...,2_CCLF_AA1005T_CM_p11_HGM3CADXY.2.aligned.dupl...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
3,CCLF_AA1005-Tumor-SM-GL7LE,CCLF_AA1005,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,CCLF_AA1005T_OPAC_p11,TSCA Rapid Cancer Detection Panel v2,SM-GL7LE,SM-GIVWM,Tumor,PCR,DNA:DNA Somatic,"Cells:Cell Line, Viable",CCLF_AA1005T_OPAC_p11,Metastatic,TM,2_CCLF_AA1005T_OPAC_p11_HGM3CADXY.2.aligned.du...,2_CCLF_AA1005T_OPAC_p11_HGM3CADXY.2.aligned.du...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
4,CCLF_AA1024-Tumor-SM-GL7LL,CCLF_AA1024,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,CCLF_AA1024T_PF_OPAC_p3,TSCA Rapid Cancer Detection Panel v2,SM-GL7LL,SM-GIR6Y,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,CCLF_AA1024T_PF_OPAC_p3,Metastatic,TM,2_CCLF_AA1024T_PF_OPAC_p3_HGM3CADXY.2.aligned....,2_CCLF_AA1024T_PF_OPAC_p3_HGM3CADXY.2.aligned....,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
5,CCLF_AA1001-Tumor-SM-GL7N2,CCLF_AA1001,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,CCLF_AA1032T_ASC_Primary,TSCA Rapid Cancer Detection Panel v2,SM-GL7N2,SM-GIR8F,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,CCLF_AA1032T_ASC_Primary,Metastatic,TM,2_CCLF_AA1032T_ASC_Primary_HGM3CADXY.2.aligned...,2_CCLF_AA1032T_ASC_Primary_HGM3CADXY.2.aligned...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
6,CCLF_AA1038-Tumor-SM-GL7NJ,CCLF_AA1038,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,CCLF_AA1038T_Primary,TSCA Rapid Cancer Detection Panel v2,SM-GL7NJ,SM-GIR8W,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,CCLF_AA1038T_Primary,Primary,TP,2_CCLF_AA1038T_Primary_HGM3CADXY.2.aligned.dup...,2_CCLF_AA1038T_Primary_HGM3CADXY.2.aligned.dup...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
7,CCLF_AA1040-Tumor-SM-GL7MW,CCLF_AA1040,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,CCLF_AA1040T_Primary,TSCA Rapid Cancer Detection Panel v2,SM-GL7MW,SM-GIR8A,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,CCLF_AA1040T_Primary,Primary,TP,2_CCLF_AA1040T_Primary_HGM3CADXY.2.aligned.dup...,2_CCLF_AA1040T_Primary_HGM3CADXY.2.aligned.dup...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
8,CCLF_AB1004-Tumor-SM-GL7M3,CCLF_AB1004,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,CCLF_AB1004T_OPAC_p4,TSCA Rapid Cancer Detection Panel v2,SM-GL7M3,SM-GIR7G,Tumor,PCR,DNA:DNA Somatic,Tissue:Fresh Tissue,CCLF_AB1004T_OPAC_p4,Metastatic,TM,2_CCLF_AB1004T_OPAC_p4_HGM3CADXY.2.aligned.dup...,2_CCLF_AB1004T_OPAC_p4_HGM3CADXY.2.aligned.dup...,gs://fc-35446f22-ea37-483a-bd6c-5e9fc56851ff/s...,TSCA22
9,

'/xchip/clf/seq_data/processed_for_fh/tsca22_201710_SN0133069/tsca22_201710_SN0133069.import_samples.txt'